In [6]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from GptCOTPrompter_BeamSeach import *
from AutoReasoner import *
import dotenv

config = dotenv.dotenv_values(".env")
openai.api_type = 'azure'
openai.api_base = 'https://meta-prompter-az-openai.openai.azure.com'
openai.api_version = '2022-12-01'
openai.api_key = config['OPENAI_API_KEY_ms']

dataset = pd.read_csv('./dataset/Table-Fact-Checking/small_test.csv', sep=',')
# dataset = pd.read_csv('./dataset/Table-Fact-Checking/small_test_sample10.csv', sep=',')
# dataset = pd.read_csv('./dataset/Table-Fact-Checking/train_sample.csv', sep=',')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')

ft = None

In [2]:
# import fasttext
# ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

In [3]:
NNDemo = False
# max_demo = 5
max_demo = 7
# template = 'original-sql'
template = 'original-sql-py-no-intermediate'
# template = 'original-sql-py-no-intermediate-v1'
# template = 'formatv1'

# demo_file = f'few-shot-demo/TabFact-formatv1.json'
# demo_file = f'few-shot-demo/TabFact-formatv2.json'
demo_file = f'few-shot-demo/TabFact-formatv3.json'
# demo_file = f'../WikiTableQuestions/few-shot-demo/WikiTQ-sql-py.json'

gpt_model = 'mp-aoi-codex'

def parallel_codex_func(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_HighTemperaturMajorityVote(
                                              f'./prompt_template/{template}.json',
                                              # '/mnt/text2sql/dataset/Table-Fact-Checking/prompt_template/formatv1.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              './data/all_csv/' + dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'], 
                                              base_path='./dataset/Table-Fact-Checking/',
                                              demo_file=demo_file,
                                             sep='#'
                                             )
            codex_prompter.model = gpt_model
            codex_prompter.max_demo = max_demo
            
            # codex_prompter._get_gpt_prediction_majority_vote(repeat_times=5)
            codex_prompter._get_gpt_prediction_majority_vote(repeat_times=5, maintain_df_ids=True)
            
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            max_retry -= 1
    return log

for program in [ 'sql', ]:
    n_threads = 2
    maxLimit = float('inf')
    # maxLimit = 1
    from joblib import Parallel, delayed
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(f'./dataset/Table-Fact-Checking/results/CodexAnswerCOTExecutor_MajorityVote_{template}_{program}_NNDemo={NNDemo}_results_test_small_limit{maxLimit}_model{gpt_model}.json', 'w'), indent=4)
    correct_cnt = 0
    for idx, l in enumerate(logs):
        if 'predicted_value' in l and l['target_value'] == l['predicted_value']:
            correct_cnt += 1
        else:
            print(idx)
    print(f"Acc = {correct_cnt} / {len(logs)} = {correct_cnt / len(logs)}")

  3%|██████▊                                                                                                                                                                                                                                    | 58/1998 [04:08<2:19:23,  4.31s/it]

Error: string indices must be integers
Error: string indices must be integers
Error: string indices must be integers


  8%|██████████████████                                                                                                                                                                                                                        | 154/1998 [11:53<2:38:03,  5.14s/it]

357


 16%|█████████████████████████████████████                                                                                                                                                                                                     | 316/1998 [25:33<2:09:19,  4.61s/it]

...

Connecting to SQLite for execution code: 
CREATE TABLE DF0_tmp AS SELECT * FROM DF0;
UPDATE DF0_tmp SET score=REPLACE(score, ' - ', ':');
UPDATE DF0_tmp SET result=REPLACE(result, ' - ', ':');.
Connecting to SQLite for execution code: 
CREATE TABLE DF0_tmp AS SELECT * FROM DF0;
UPDATE DF0_tmp SET score=REPLACE(score, ' - ', ':');
UPDATE DF0_tmp SET result=REPLACE(result, ' - ', ':');.
Connecting to SQLite for execution code: 
CREATE TABLE DF0_tmp AS SELECT * FROM DF0;
UPDATE DF0_tmp SET score=REPLACE(score, ' - ', ':');
UPDATE DF0_tmp SET result=REPLACE(result, ' - ', ':');.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 1996/1998 [2:42:54<00:08,  4.45s/it]

Connecting to SQLite for execution code: SELECT ABS(SUBSTR(score, 1, 1) - SUBSTR(score, -1, 1)) FROM DF0 WHERE date="february 10";.
Connecting to SQLite for execution code: SELECT ABS(SUBSTR(score, 1, 1) - SUBSTR(score, -1, 1)) FROM DF0 WHERE date="february 10";.
Connecting to SQLite for execution code: SELECT ABS(SUBSTR(score, 1, 1) - SUBSTR(score, -1, 1)) FROM DF0 WHERE date="february 10";.
Connecting to SQLite for execution code: SELECT ABS(SUBSTR(score, 1, 1) - SUBSTR(score, -1, 1)) FROM DF0 WHERE date="february 10";.
Connecting to SQLite for execution code: SELECT ABS(SUBSTRING_INDEX(SUBSTRING_INDEX(score, " - ", 1), " - ", -1) - SUBSTRING_INDEX(SUBSTRING_INDEX(score, " - ", 2), " - ", -1)) FROM DF0 WHERE date="february 10" OR date="february 24";.
Connecting to SQLite for execution code: SELECT ABS(SUBSTRING_INDEX(SUBSTRING_INDEX(score, " - ", 1), " - ", -1) - SUBSTRING_INDEX(SUBSTRING_INDEX(score, " - ", 2), " - ", -1)) FROM DF0 WHERE date="february 10" OR date="february 24";.
Co

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1998/1998 [2:43:02<00:00,  4.90s/it]


1
4
6
10
14
21
26
28
29
43
47
50
57
59
64
72
79
80
82
91
102
109
112
115
117
119
124
128
133
135
140
146
156
160
164
172
174
181
184
185
186
187
191
194
201
204
208
213
214
218
227
229
234
240
244
246
249
250
257
259
260
263
280
281
285
289
297
310
313
317
325
328
335
338
343
363
368
398
401
415
419
425
429
437
440
442
448
449
457
460
468
470
472
474
477
482
484
501
504
505
511
513
514
519
525
533
534
535
537
543
545
546
547
561
571
572
576
582
583
584
586
589
591
594
596
599
600
607
609
612
616
618
623
635
642
644
655
656
657
664
666
671
672
682
683
688
694
702
706
709
710
712
721
724
730
731
738
740
752
760
763
764
769
787
789
802
811
812
816
818
821
826
831
833
838
840
842
848
853
855
858
868
874
875
876
895
914
915
917
920
927
930
933
934
947
951
960
963
968
983
988
1003
1004
1011
1013
1018
1023
1030
1033
1035
1037
1039
1040
1041
1044
1046
1048
1062
1071
1074
1079
1083
1084
1092
1095
1096
1100
1104
1106
1108
1135
1136
1140
1144
1150
1152
1154
1157
1159
1160
1164
1165
1168
1171
1178

In [4]:
logs[0]

{'id': 'nt-17418',
 'utterance': 'neil labute be nominate for best play in 2009. yes or no?',
 'source_csv': './dataset/Table-Fact-Checking/./data/all_csv/2-18963715-1.html.csv',
 'target_value': 'yes',
 'predicted_value': 'yes',
 'prompt': 'The database table DF is shown as follows:\n[HEAD]: year|venue|winner|country|score\n---\n[ROW] 1: 2005|arlandastad|david patrick|scotland|272\n[ROW] 2: 2004|arlandastad|matthew king|england|270\n[ROW] 3: 2003|falsterbo|titch moore|south africa|273\n[ROW] 4: 2002|halmstad|thomas besancenez|france|279\n[ROW] 5: 2001|bokskogens|cristophe pottier|france|272\n\nJudge whether the following statement is correct or not: "in 2004 the score be less than 270. yes or no?". Generate SQL or Python code step-by-step given the question and table to answer the question correctly. For each step, generate SQL code to process the query or Python code to reformat the data. Output the code braced by "```" and an external executor will process the code generated and fee

In [7]:
def parallel_codex_func(i):

    codex_prompter = CodexAnswerCOTExecutor_template(
                                      f'/mnt/text2sql/dataset/Table-Fact-Checking/prompt_template/{template}.json',
                                      # '/mnt/text2sql/dataset/Table-Fact-Checking/prompt_template/formatv1.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      './data/all_csv/' + dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'], 
                                      base_path='./dataset/Table-Fact-Checking/',
                                      demo_file=f'few-shot-demo/TabFact-formatv1.json',
                                     sep='#'
                                     )
    codex_prompter.model = gpt_model
    codex_prompter.max_demo = max_demo
    # codex_prompter._gen_gpt_prompt()
    codex_prompter._gen_gpt_prompt()
    codex_prompter._get_gpt_prediction()
    log = codex_prompter._log_dict()
    return log
parallel_codex_func(0)

{'id': 'nt-17418',
 'utterance': 'neil labute be nominate for best play in 2009. yes or no?',
 'source_csv': './dataset/Table-Fact-Checking/./data/all_csv/2-18963715-1.html.csv',
 'target_value': 'yes',
 'predicted_value': 'yes',
 'prompt': 'The database table DF is shown as follows:\n[HEAD]: year|venue|winner|country|score\n---\n[ROW] 1: 2005|arlandastad|david patrick|scotland|272\n[ROW] 2: 2004|arlandastad|matthew king|england|270\n[ROW] 3: 2003|falsterbo|titch moore|south africa|273\n[ROW] 4: 2002|halmstad|thomas besancenez|france|279\n[ROW] 5: 2001|bokskogens|cristophe pottier|france|272\n\nAnswer the following question based on the data above: "in 2004 the score be less than 270. yes or no?". Execute SQL  code step-by-step and finally answer the question. Choose from generating a SQL code or directly answering the question.\n\nSQL: ```SELECT score FROM DF WHERE year=2004;```.\n\nThe database table DF is shown as follows:\n[HEAD]: score\n---\n[ROW] 1: 270\n\nAnswer the following qu